# Simulação de comunicação digital com canal ruidoso
## Visão Geral do Projeto
**Objetivo:** Criar uma simulação de comunicação digital com canal ruidoso, onde o professor possa ajustar parâmetros como taxa de erro, intensidade de ruído, tipo de modulação, e visualizar impactos no desempenho do sistema de comunicação, incluindo métricas como Taxa de Erro de Bit (BER), Sinal Ruído (SNR), etc.

### Ferramentas
- **Plotly** para gráficos interativos.
- **ipywidget** para interação com sliders e controles.
- **NumPy** e **SciPy** para cálculos e geração de sinais.
- **Jupyter Notebook** (para protótipo inicial), e eventualmente transformar em aplicativo.
- **Streamlit** para, posteriormente, subir o aplicativo para produção. 

## Configurações básicas
Instalação das bibliotecas necessárias para o projeto.
Para uma maneira automatizada:

In [ ]:
pip install requirements.txt

Ou, de maneira manual:

In [ ]:
pip install plotly ipywidgets numpy scipy

## Geração de Sinais e Canal (BWSG)
Para a simulação, será gerado e plotado um sinal binário no domínio do tempo e aplicando um canal com ruído Gaussiano (como o BSWG).

### Gerando o Sinal Binário
O sinal binário é simplesmente uma sequência e 0s e 1s que representam os bits sendo transmitidos.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Gerando um sinal binário aleatório de 100 bits
n_bits = 100
rng = np.random.default_rng()
sinal = rng.integers(0, 2, n_bits)

# Plotando o sinal
figura = go.Figure(data=go.Scatter(x=np.arange(n_bits), y=sinal, mode='lines', name='Sinal Binário'))
figura.update_layout(title="Sinal Binário Transmitido", xaxis_title="Tempo", yaxis_title="Valor do Sinal")
figura.show()

### Adicionando Ruído Gaussiano (Canal BWSG)
Adiciona-se, então, o ruído Gaussiano ao sinal, para simular a distorção que ocorre em um canal de comunicação ruidoso.

In [ ]:
def adicionar_ruido(sinal, desvio_padrao):
    ruido = np.random.normal(0, desvio_padrao, len(sinal))
    sinal_ruidoso = sinal + ruido
    
    return sinal_ruidoso

# Parâmetro de desvio padrão para o ruído (intensidade do ruído)
desvio_padrao = 0.5
sinal_ruidoso = adicionar_ruido(sinal, desvio_padrao)

# Plotando os sinais
figure = go.Figure()
figure.add_trace(go.Scatter(x=np.arange(n_bits), y=sinal, mode='lines', name='Sinal Original'))
figure.add_trace(go.Scatter(x=np.arange(n_bits), y=sinal_ruidoso, mode='lines', name='Sinal com Ruído'))
figure.update_layout(title="Sinal Original e Sinal Ruidoso", xaxis_title="Tempo", yaxis_title="Valor do Sinal")
figure.show()

## Interatividade com os parâmetros do canal
Torna os parâmetros utilizados do canal em algo variável/interativo.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Função interativa para atualizar o gráfico conforme a mudança do ruído
def atualiza_grafico(desvio_padrao):
    sinal_ruidoso = adicionar_ruido(sinal, desvio_padrao)

    # Plotando os sinais interativos
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(n_bits), y=sinal, mode='lines', name='Sinal Original'))
    fig.add_trace(go.Scatter(x=np.arange(n_bits), y=sinal_ruidoso, mode='lines', name='Sinal com Ruído'))
    fig.update_layout(title=f"Sinal Original e Sinal Ruidoso (Desvio Padrão = {desvio_padrao})", 
                      xaxis_title="Tempo", yaxis_title="Valor do Sinal")
    fig.show()
    
# Slider interativo para o desvio padrão do ruído
slider_ruido = widgets.FloatSlider(min=0, max=3, step=0.1, value=0.5)
widgets.interactive(atualiza_grafico, desvio_padrao=slider_ruido)

## Análise de Desempenho
A partir daqui é possível identificar, através de vários indicadores, as diferenças entre o sinal original e o mesmo sinal, agora ruidoso.

### Taxa de Erro de Bit (BER) - Cálculo da BER
Para calcular a BER, primeiro é preciso definir a decodificação do sinal. Supõe-se, então, que o receptor aplique uma medida para decidir se o bit transmitido é 0 ou 1 com base no valor do sinal recebido.

In [ ]:
def calcula_ber(sinal_original, sinal_recebido):
    # Decodificação simples: Se o valor recebido for maior que 0.5, o bit é 1, senão é 0
    sinal_decodificado = np.where(sinal_recebido > 0.5, 1, 0)
    
    # Calculando erros (diferença entre sinal transmitido e recebido)
    erros = np.sum(sinal_decodificado != sinal_original)
    ber = erros / len(sinal_original)
    return ber

# Cálculo do BER para o sinal original e o sinal ruidoso
ber = calcula_ber(sinal, sinal_ruidoso)
print(f"Taxa de Erro de Bit (BER): {ber:.4f}")

### Sinal-Ruído (SNR) - Cálculo do SNR
O SNR (Sinal-Ruído) pode ser calculado pela razão entre a potência do sinal e a potência do ruído.

In [ ]:
def calcula_snr(sinal_original, sinal_ruidoso):
    potencia_sinal = np.mean(sinal_original**2)
    potencia_ruido = np.mean((sinal_ruidoso - sinal_original)**2)
    snr = 10 * np.log10(potencia_sinal / potencia_ruido)
    return snr

# Cálculo do SNR para o sinal original e ruidoso
snr = calcula_snr(sinal, sinal_ruidoso)
print(f"SNR (Sinal-Ruído): {snr:.2f} dB")

# Conclusão
Esse projeto explorar como o ruído afeta a transmissão de sinais e como isso impacta o desempenho de um sistema de comunicação.